In [21]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
emb_model = SentenceTransformer('KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS')

# Tokenize input texts
# texts = [
#     "저기 스케이트보드를 타는 학생이 있다.",
#     "한 남자가 스케이트보드를 탄다.",
#     "할아버지가 식사를 하신다."
# ]

# vectors = model.encode(texts)
# similarities = cosine_similarity(vectors)
# print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], similarities[0][1]))
# print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], similarities[0][2]))

# 감성 분석 과정
0. 데이터를 다운 받는다
1. document랑 label을 분리한다.
2. document를 모델에 넣어서 임베딩 벡터로 바꾼다.
3. 신경망 모델을 만든다.
 모델로 데이터를 모두 임베딩을 한다.

In [2]:
import pandas as pd

In [3]:
PATH = './nsmc/'
ratings_train = pd.read_table(PATH + 'ratings_train.txt') 
ratings_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
ratings_test = pd.read_table(PATH + 'ratings_test.txt') 
ratings_test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [5]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [6]:
ratings_train['document'][:5]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [7]:
train_copy = ratings_train.copy()
test_copy = ratings_test.copy()

In [8]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [9]:
train_copy=train_copy.dropna(axis=0).reset_index(drop=True)
test_copy=test_copy.dropna(axis=0).reset_index(drop=True)

In [10]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149995 entries, 0 to 149994
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [11]:
train_copy.index

RangeIndex(start=0, stop=149995, step=1)

In [12]:
from tqdm import tqdm
for i in tqdm(range(len(train_copy))):
  try:
    train_copy.at[i, 'document'] = clean(train_copy.at[i, 'document'])
  except KeyError as e:
    print(e)
    print(train_copy.at[i, 'document'])  

100%|██████████| 149995/149995 [00:01<00:00, 94051.48it/s]


In [13]:
test_copy.at[5746, 'document']

'지금 봐도 모자람이 없게 느껴지는 연출과 그래픽. 15년 대장정의 멋진 시작!'

In [14]:
for i in tqdm(range(len(test_copy))):
  try:
    test_copy.at[i, 'document'] = clean(test_copy.at[i, 'document'])
  except KeyError as e:
    print(e)
    print(test_copy.at[i, 'document'])

100%|██████████| 49997/49997 [00:00<00:00, 91249.28it/s]


In [15]:
#train_x = ratings_train['document']
#train_x
# train_y = ratings_train['label']
# print(train_x.shape, train_y.shape)

In [16]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train_copy['document'], train_copy['label'], test_size = 0.2, random_state = 0)

In [17]:
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

(119996,) (119996,)
(29999,) (29999,)


In [18]:
test_x = test_copy ['document']
test_y = test_copy['label']
print(test_x.shape, test_y.shape)

(49997,) (49997,)


In [22]:
train_vectors_X = emb_model.encode(train_x.values)
val_vectors_X = emb_model.encode(val_x.values)
test_vectors_X = emb_model.encode(test_x)

In [23]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
# dense -> batchnorm -> activation -> drop out
# 256->128->64->32->1
model = models.Sequential()
model.add(layers.Dense(256, input_shape=(768,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(layers.Dropout(0.5))

# model.add(layers.Dense(32))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(layers.Dropout(0.5))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.Adam(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

checkpoint = ModelCheckpoint(PATH, monitor='val_loss', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)


model.fit(train_vectors_X, train_y, validation_data=(val_vectors_X, val_y), epochs=1000, batch_size=512, callbacks=[checkpoint, earlystopping])


2021-11-04 21:21:44.794273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 21:21:44.823177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 21:21:44.823665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 21:21:44.824658: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/1000
 53/235 [=====>........................] - ETA: 0s - loss: 0.4982 - binary_accuracy: 0.7639

2021-11-04 21:21:46.419023: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


235/235 [==============================] - 2s 4ms/step - loss: 0.4105 - binary_accuracy: 0.8177 - val_loss: 0.3520 - val_binary_accuracy: 0.8469

Epoch 00001: val_loss improved from inf to 0.35204, saving model to ./nsmc/


2021-11-04 21:21:47.609396: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./nsmc/assets
Epoch 2/1000
235/235 [==============================] - 1s 3ms/step - loss: 0.3526 - binary_accuracy: 0.8496 - val_loss: 0.3366 - val_binary_accuracy: 0.8528

Epoch 00002: val_loss improved from 0.35204 to 0.33660, saving model to ./nsmc/
INFO:tensorflow:Assets written to: ./nsmc/assets
Epoch 3/1000
235/235 [==============================] - 1s 3ms/step - loss: 0.3384 - binary_accuracy: 0.8574 - val_loss: 0.3199 - val_binary_accuracy: 0.8621

Epoch 00003: val_loss improved from 0.33660 to 0.31994, saving model to ./nsmc/
INFO:tensorflow:Assets written to: ./nsmc/assets
Epoch 4/1000
235/235 [==============================] - 1s 3ms/step - loss: 0.3277 - binary_accuracy: 0.8610 - val_loss: 0.3193 - val_binary_accuracy: 0.8623

Epoch 00004: val_loss improved from 0.31994 to 0.31930, saving model to ./nsmc/
INFO:tensorflow:Assets written to: ./nsmc/assets
Epoch 5/1000
235/235 [==============================] - 1s 3ms/step - loss: 0.3198 - bi

In [24]:
from tensorflow.keras.models import load_model

best_model = load_model(PATH)
results = best_model.evaluate(test_vectors_X, test_y)

1563/1563 [==============================] - 2s 925us/step - loss: 0.3016 - binary_accuracy: 0.8728


In [25]:
from utils.data_frame import Data_frame

In [26]:
df = Data_frame('마켓컬리', '리뷰').get_DF()
df.head()

,name,user,star,content
0,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,황*현,,지난번 괜찮었던 기억으로 재구매 했는데 다른 브랜드였나 싶게 기름이 많어도 늠~ 많아요
1,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,조*정,,이전에 먹었는데 맛있어서 재구매합니다. 전에는 고기가 두꺼운데 팬에 굽느라 냄새도 ...
2,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,유*,,정망 맛있게 먹었어요 부모님도 좋아하시네요~
3,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,남*인,,더니 너무 잘 먹어 또 쥬문요
4,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,이*주,,맛은 괜찮아요 근데 지방이 좀 많네요. 에어프라이어 돌려 드세요.


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16481 entries, 0 to 16480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     16481 non-null  object
 1   user     16481 non-null  object
 2   star     16481 non-null  object
 3   content  16481 non-null  object
dtypes: object(4)
memory usage: 515.2+ KB


In [31]:
df

,name,user,star,content,emb_content
0,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,황*현,,지난번 괜찮었던 기억으로 재구매 했는데 다른 브랜드였나 싶게 기름이 많어도 늠~ 많아요,0
1,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,조*정,,이전에 먹었는데 맛있어서 재구매합니다. 전에는 고기가 두꺼운데 팬에 굽느라 냄새도 ...,0
2,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,유*,,정망 맛있게 먹었어요 부모님도 좋아하시네요~,0
3,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,남*인,,더니 너무 잘 먹어 또 쥬문요,0
4,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,이*주,,맛은 괜찮아요 근데 지방이 좀 많네요. 에어프라이어 돌려 드세요.,0
...,...,...,...,...,...
16476,[낭만어시장] 얼큰 알탕,한*주,,"2명이서 먹기엔 조금 양이 적고, 추가로 두부나 쑥갓 대파를 더 넣으니 좋습니다~!!",0
16477,[낭만어시장] 얼큰 알탕,유*연,,국물이 시원합니다 구성도 다양하구요 맛있어요,0
16478,[낭만어시장] 얼큰 알탕,심*희,,보고 샀는데 아직 먹어보진 않았어요 알탕 좋아해서 맛있게 잘 먹을 것 같아요,0
16479,[낭만어시장] 얼큰 알탕,손*권,,낭만어시장알탕 잘받았습니다~~,0


In [32]:
# df['emb_content'] = 0
for i in tqdm(range(len(df))):
    df.at[i, 'content'] = clean(df.at[i, 'content'])

100%|██████████| 16481/16481 [00:00<00:00, 94926.75it/s]


In [33]:
review_vector = emb_model.encode(df['content'].values)
review_vector[:3]

array([[-0.3646573 , -0.3251074 , -1.1858953 , ...,  0.35351303,
        -0.14340854,  0.24134117],
       [ 0.04697471, -0.08881704, -0.72056055, ...,  0.05675216,
        -0.35330698,  0.19085896],
       [-1.0835663 , -0.8103261 ,  0.22853942, ..., -0.6862127 ,
        -0.28821766, -0.208606  ]], dtype=float32)

In [40]:
pred = best_model.predict(review_vector)
pred[:5]

array([[0.9344138 ],
       [0.57465225],
       [0.973011  ],
       [0.9813239 ],
       [0.04840476]], dtype=float32)

In [37]:
sentiment = [1 if pred > 0.5 else 0 for pred in pred]
sentiment[:5]

[1, 1, 1, 1, 0]

In [39]:
df['star'] = sentiment
df[:30]

,name,user,star,content,emb_content
0,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,황*현,1,지난번 괜찮었던 기억으로 재구매 했는데 다른 브랜드였나 싶게 기름이 많어도 늠~ 많아요,0
1,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,조*정,1,이전에 먹었는데 맛있어서 재구매합니다. 전에는 고기가 두꺼운데 팬에 굽느라 냄새도 ...,0
2,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,유*,1,정망 맛있게 먹었어요 부모님도 좋아하시네요~,0
3,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,남*인,1,더니 너무 잘 먹어 또 쥬문요,0
4,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,이*주,0,맛은 괜찮아요 근데 지방이 좀 많네요. 에어프라이어 돌려 드세요.,0
5,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,류*정,1,기름기가 많아 그런지 고기가 맛은 좋아요. 남편은 느끼하다는데 퍽퍽한 고기보다 훨씬...,0
6,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,이*애,0,"너무 퍽퍽한 느낌..,ㅜ",0
7,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,임*수,1,여름엔 배달로 버티다 선선해져서 에프좀 키려구요ㅎ 요거 맛있는데 에프에 해도 느끼함...,0
8,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,양*숙,0,육질은 좋아요 비계가 좀 많아요,0
9,[르블란서] 돈마호크 스테이크&머스터드 소스 세트,조*혜,1,기름이 너무 많아요ㅜㅜ,0
